## 퍼셉트론

```
     - 신경망을 이루는 가장 중요한 기본 단위는 퍼셉트론
     - 퍼셉트론 : 입력 값과 활성화 함수를 사용해 출력 값을 다음으로 넘기는 가장 작은 신경망 단위
     
     - 기울기 = 가중치 (w), y절편 = 바이어스 (b)
     - 가중합의 결과를 놓고 1또는 0을 출력해서 다음으로 보냄
     - 활성화 함수를 통해 1과 0을 판단해 출력함
     
```

###  Single neural network AND 연산

In [1]:
import numpy as np
import tensorflow as tf

In [ ]:
x_data = np.array([[0,0],[0,1],[1,0],[1,1]], dtype=np.float32)
y_data = np.array([[0],[1],[1],[0]], dtype=np.float32)

X = tf.placeholder(tf.float32, [None,2],name='x_input')
Y = tf.placeholder(tf.float32, [None,1],name='y_input')

#가중치
W = tf.Variable(tf.random_normal([2,1], name='weight'))
#바이어스
b = tf.Variable(tf.random_normal([1]),name='bias')

#활성화 함수
hypothesis = tf.sigmoid(tf.matmul(X,W)+b)

#cost/loss
cost = -tf.reduce_mean(Y * tf.log(hypothesis) + (1 - Y) *tf.log(1 - hypothesis))
train = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)

#예측값 결과 계산 및 정밀도 계산
predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, Y), dtype = tf.float32))


#Launch graph
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for step in range(10001):
        sess.run(train,feed_dict={X:x_data,Y:y_data})
        
        if step % 100 == 0:
            print("step = ",step,"cost= ",sess.run(cost,feed_dict={X:x_data,Y:y_data}), "W= ", sess.run(W), "b= ", sess.run(b))
            
    h,c,a=sess.run([hypothesis,predicted,accuracy], feed_dict ={X:x_data, Y:y_data})
    print("\nHypothesis:",h,"\nCorrect(y): ",c,"\n Accuracy:",a)
            
    


## 다중 퍼셉트론 [ XOR ]

```  
    1. 따라서 만약 두개의 직선(다중)을 만들기 위해서 두개의 뉴런이 필요[hidden layer]
    2. 이 둘을 교차시키는 연산이 필요함. 
    3. 여기서 활성함수로는 시그모이드를 사용함. 
    
    
    * 여기서 AND,OR,NAND를 (hidden layer로 구성되는) 만족하는 가중치와 바이어스의 조합은 무수히 많음.
    
    y = w1*1 _ w2*2 + b
    
 
```

In [ ]:
################   NUMPY로 짜보기   ############


#가중치와 바이어스
w11 = np.array([-7.40,-7.40])
w12 = np.array([8.67,8.67])
w2 = np.array([7.41,7.41])
b1 = 11.28
b2 = -2.87
b3 = -11.29

#퍼센트론
def MLP(x ,w ,b):
    y = np.sum(w*x) + b
    if y <= 0:
        return 0
    else:
        return 1
    
    
#NAND 게이트
def NAND(x1, x2):
    return MLP(np.array([x1,x2]),w11,b1)

def OR(x1, x2):
    return MLP(np.array([x1, x2]), w12, b2)

#AND 게이트
def AND(x1, x2):
    return MLP(np.array([x1, x2]), w2,b3)

def XOR(x1, x2):
    return AND(NAND(x1, x2), OR(x1,x2))


#x1, x2 값을 번갈아 대입해 가며 최종값 출력
if __name__ == '__main__':
    for x in [(0,0),(1,0),(0,1),(1,1)]:
        y = XOR(x[0],x[1])
        print("입력값 : "+ str(x) + "출력값:" + str(y))



## 오차 역전파

```
    신경망 내부의 가중치는 오차 역전파 방법을 사용해 수정
    * 가중치  --> 경사 하강법 사용!
                  실제값(y)이 나오면 출력층에서 시작해 뒤에서 앞(hidden layer 쪽)으로 진행됨 
                  이과정에서 가중치에서 기울기를 뺏을 때 가중치 변화가 없을 때까지 나아감.
                  --> 오차 역전파
    
```

In [ ]:
learning_rate = 0.1
tf.set_random_seed(0)
np.random.seed(0)

x_data = [[0,0],[0,1],[1,0],[1,1]]
y_data=[[0],[1],[1],[0]]

X = tf.placeholder(tf.float32,[None,2])
Y = tf.placeholder(tf.float32,[None,1])

W1 = tf.Variable(tf.random_normal([2,2], name ='weight1'))
b1 = tf.Variable(tf.random_normal([2]), name = 'bias1')
layer1 = tf.sigmoid(tf.matmul(X,W1) + b1)   #hideen layer

W2 = tf.Variable(tf.random_normal([2,1]),name='weight2')
b2 = tf.Variable(tf.random_normal([1]),name='bias2')
hypothesis = tf.sigmoid(tf.matmul(layer1,W2) + b2)

cost = -tf.reduce_mean(Y*tf.log(hypothesis) + (1-Y) *tf.log(1-hypothesis))
train = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted,Y),dtype=tf.float32))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for step in range(10001):
        sess.run(train,feed_dict={X:x_data, Y:y_data})
        if step % 100 == 0:
            print(step, sess.run(cost, feed_dict={X:x_data,Y:y_data}), sess.run(W2))
            
    h,c,a = sess.run([hypothesis, predicted, accuracy], feed_dict={X:x_data, Y:y_data})
    print("\nHypothesis:\n",h, "\nCorrect:\n",c ,"\nAccuracy:\n",a)


In [ ]:
## 노드를 늘려보기 ###


learning_rate = 0.1
tf.set_random_seed(0)
np.random.seed(0)

x_data = [[0,0],[0,1],[1,0],[1,1]]
y_data=[[0],[1],[1],[0]]

X = tf.placeholder(tf.float32,[None,2])
Y = tf.placeholder(tf.float32,[None,1])

W1 = tf.Variable(tf.random_normal([2,5], name ='weight1'))
b1 = tf.Variable(tf.random_normal([5]), name = 'bias1')
layer1 = tf.sigmoid(tf.matmul(X,W1) + b1)   #hideen layer

W2 = tf.Variable(tf.random_normal([5,1]),name='weight2')
b2 = tf.Variable(tf.random_normal([1]),name='bias2')
hypothesis = tf.sigmoid(tf.matmul(layer1,W2) + b2)

cost = -tf.reduce_mean(Y*tf.log(hypothesis) + (1-Y) *tf.log(1-hypothesis))
train = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted,Y),dtype=tf.float32))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for step in range(10001):
        sess.run(train,feed_dict={X:x_data, Y:y_data})
        if step % 100 == 0:
            print(step, sess.run(cost, feed_dict={X:x_data,Y:y_data}), sess.run(W2))
            
    h,c,a = sess.run([hypothesis, predicted, accuracy], feed_dict={X:x_data, Y:y_data})
    print("\nHypothesis:\n",h, "\nCorrect:\n",c ,"\nAccuracy:\n",a)

In [ ]:
### hidden layer 늘려보기 ###
learning_rate = 0.1
tf.set_random_seed(0)
np.random.seed(0)

x_data = [[0,0],[0,1],[1,0],[1,1]]
y_data=[[0],[1],[1],[0]]

X = tf.placeholder(tf.float32,[None,2])
Y = tf.placeholder(tf.float32,[None,1])

W1 = tf.Variable(tf.random_normal([2,5], name ='weight1'))
b1 = tf.Variable(tf.random_normal([5]), name = 'bias1')
layer1 = tf.sigmoid(tf.matmul(X,W1) + b1)   #hideen layer

W2 = tf.Variable(tf.random_normal([5,5], name ='weight2'))
b2 = tf.Variable(tf.random_normal([5]), name = 'bias2')
layer2 = tf.sigmoid(tf.matmul(layer1,W2) + b2)   #hideen layer

W3 = tf.Variable(tf.random_normal([5,5]),name='weight3')
b3 = tf.Variable(tf.random_normal([5]),name='bias3')
layer3= tf.sigmoid(tf.matmul(layer2,W3) + b3)

W4 = tf.Variable(tf.random_normal([5,1]),name='weight4')
b4 = tf.Variable(tf.random_normal([5]),name='bias4')
hypothesis = tf.sigmoid(tf.matmul(layer3,W4) + b4)

cost = -tf.reduce_mean(Y*tf.log(hypothesis) + (1-Y) *tf.log(1-hypothesis))
train = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted,Y),dtype=tf.float32))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for step in range(10001):
        sess.run(train,feed_dict={X:x_data, Y:y_data})
        if step % 100 == 0:
            print(step, sess.run(cost, feed_dict={X:x_data,Y:y_data}), sess.run(W2))
            
    h,c,a = sess.run([hypothesis, predicted, accuracy], feed_dict={X:x_data, Y:y_data})
    print("\nHypothesis:\n",h, "\nCorrect:\n",c ,"\nAccuracy:\n",a)

In [ ]:
### hidden layer 늘려보기 ###
learning_rate = 0.1
tf.set_random_seed(0)
np.random.seed(0)

x_data = [[0,0],[0,1],[1,0],[1,1]]
y_data=[[0],[1],[1],[0]]

X = tf.placeholder(tf.float32,[None,2])
Y = tf.placeholder(tf.float32,[None,1])

W1 = tf.Variable(tf.random_normal([2,5], name ='weight1'))
b1 = tf.Variable(tf.random_normal([5]), name = 'bias1')
layer1 = tf.sigmoid(tf.matmul(X,W1) + b1)   #hideen layer

W2 = tf.Variable(tf.random_normal([5,5], name ='weight2'))
b2 = tf.Variable(tf.random_normal([5]), name = 'bias2')
layer2 = tf.sigmoid(tf.matmul(layer1,W2) + b2)   #hideen layer

W3 = tf.Variable(tf.random_normal([5,5]),name='weight3')
b3 = tf.Variable(tf.random_normal([5]),name='bias3')
layer3= tf.sigmoid(tf.matmul(layer2,W3) + b3)

W4 = tf.Variable(tf.random_normal([5,5], name ='weight4'))
b4 = tf.Variable(tf.random_normal([5]), name = 'bias4')
layer4 = tf.sigmoid(tf.matmul(layer3,W4) + b4)   #hideen layer

W5 = tf.Variable(tf.random_normal([5,5]),name='weight5')
b5 = tf.Variable(tf.random_normal([5]),name='bias5')
layer5= tf.sigmoid(tf.matmul(layer4,W5) + b5)

W6 = tf.Variable(tf.random_normal([5,5], name ='weight6'))
b6 = tf.Variable(tf.random_normal([5]), name = 'bias6')
layer6 = tf.sigmoid(tf.matmul(layer5,W6) + b6)   #hideen layer

W7 = tf.Variable(tf.random_normal([5,5]),name='weight7')
b7 = tf.Variable(tf.random_normal([5]),name='bias7')
layer7= tf.sigmoid(tf.matmul(layer6,W7) + b7)

W8 = tf.Variable(tf.random_normal([5,5], name ='weight8'))
b8 = tf.Variable(tf.random_normal([5]), name = 'bias8')
layer8 = tf.sigmoid(tf.matmul(layer7,W8) + b8)   #hideen layer

W9 = tf.Variable(tf.random_normal([5,5]),name='weight9')
b9 = tf.Variable(tf.random_normal([5]),name='bias9')
layer9= tf.sigmoid(tf.matmul(layer8,W9) + b9)

W10 = tf.Variable(tf.random_normal([5,1]),name='weight10')
b10 = tf.Variable(tf.random_normal([5]),name='bias10')
hypothesis = tf.sigmoid(tf.matmul(layer9,W10) + b10)

cost = -tf.reduce_mean(Y*tf.log(hypothesis) + (1-Y) *tf.log(1-hypothesis))
train = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted,Y),dtype=tf.float32))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for step in range(10001):
        sess.run(train,feed_dict={X:x_data, Y:y_data})
        if step % 100 == 0:
            print(step, sess.run(cost, feed_dict={X:x_data,Y:y_data}), sess.run(W2))
            
    h,c,a = sess.run([hypothesis, predicted, accuracy], feed_dict={X:x_data, Y:y_data})
    print("\nHypothesis:\n",h, "\nCorrect:\n",c ,"\nAccuracy:\n",a)

## Relu함수

``` 
    오차 역전파는 출력층으로 하나씩 돌아가며 각 층의 가중치를 수정 
    -->기울기가 필요 
    --> 층이 늘어나면서 기울기가 중간에 0이 되어버리는 기울기 소실이 발생
    
    
    ReLU함수이용 : 입력값이 0보다 작을 떄는 아예 non-activate로 꺼버리고, 0보다 클 때는 그 값을 그대로 만환
       이 함수는 시그모이드를 대체해서 사용 단 마지막 부분은 0과 1사이로 표현하기위해 사용.
       tf.nn.relu()
    
```    

In [2]:
### hidden layer 늘려보기 ###
learning_rate = 0.1
tf.set_random_seed(0)
np.random.seed(0)

x_data = [[0,0],[0,1],[1,0],[1,1]]
y_data=[[0],[1],[1],[0]]

X = tf.placeholder(tf.float32,[None,2])
Y = tf.placeholder(tf.float32,[None,1])

W1 = tf.Variable(tf.random_normal([2,5], name ='weight1'))
b1 = tf.Variable(tf.random_normal([5]), name = 'bias1')
layer1 = tf.nn.relu(tf.matmul(X,W1) + b1)   #hideen layer

W2 = tf.Variable(tf.random_normal([5,5], name ='weight2'))
b2 = tf.Variable(tf.random_normal([5]), name = 'bias2')
layer2 = tf.nn.relu(tf.matmul(layer1,W2) + b2)   #hideen layer

W3 = tf.Variable(tf.random_normal([5,5]),name='weight3')
b3 = tf.Variable(tf.random_normal([5]),name='bias3')
layer3= tf.nn.relu(tf.matmul(layer2,W3) + b3)

W4 = tf.Variable(tf.random_normal([5,5], name ='weight4'))
b4 = tf.Variable(tf.random_normal([5]), name = 'bias4')
layer4 = tf.nn.relu(tf.matmul(layer3,W4) + b4)   #hideen layer

W5 = tf.Variable(tf.random_normal([5,5]),name='weight5')
b5 = tf.Variable(tf.random_normal([5]),name='bias5')
layer5= tf.nn.relu(tf.matmul(layer4,W5) + b5)

W6 = tf.Variable(tf.random_normal([5,1]),name='weight6')
b6 = tf.Variable(tf.random_normal([5]),name='bias6')
hypothesis = tf.sigmoid(tf.matmul(layer5,W6) + b6)

cost = -tf.reduce_mean(Y*tf.log(hypothesis) + (1-Y) *tf.log(1-hypothesis))
train = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted,Y),dtype=tf.float32))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for step in range(10001):
        sess.run(train,feed_dict={X:x_data, Y:y_data})
        if step % 100 == 0:
            print(step, sess.run(cost, feed_dict={X:x_data,Y:y_data}), sess.run(W2))
            
    h,c,a = sess.run([hypothesis, predicted, accuracy], feed_dict={X:x_data, Y:y_data})
    print("\nHypothesis:\n",h, "\nCorrect:\n",c ,"\nAccuracy:\n",a)

0 nan [[nan nan nan nan nan]
 [nan nan nan nan nan]
 [nan nan nan nan nan]
 [nan nan nan nan nan]
 [nan nan nan nan nan]]
100 nan [[nan nan nan nan nan]
 [nan nan nan nan nan]
 [nan nan nan nan nan]
 [nan nan nan nan nan]
 [nan nan nan nan nan]]
200 nan [[nan nan nan nan nan]
 [nan nan nan nan nan]
 [nan nan nan nan nan]
 [nan nan nan nan nan]
 [nan nan nan nan nan]]
300 nan [[nan nan nan nan nan]
 [nan nan nan nan nan]
 [nan nan nan nan nan]
 [nan nan nan nan nan]
 [nan nan nan nan nan]]
400 nan [[nan nan nan nan nan]
 [nan nan nan nan nan]
 [nan nan nan nan nan]
 [nan nan nan nan nan]
 [nan nan nan nan nan]]
500 nan [[nan nan nan nan nan]
 [nan nan nan nan nan]
 [nan nan nan nan nan]
 [nan nan nan nan nan]
 [nan nan nan nan nan]]
600 nan [[nan nan nan nan nan]
 [nan nan nan nan nan]
 [nan nan nan nan nan]
 [nan nan nan nan nan]
 [nan nan nan nan nan]]
700 nan [[nan nan nan nan nan]
 [nan nan nan nan nan]
 [nan nan nan nan nan]
 [nan nan nan nan nan]
 [nan nan nan nan nan]]
800 nan [[

6900 nan [[nan nan nan nan nan]
 [nan nan nan nan nan]
 [nan nan nan nan nan]
 [nan nan nan nan nan]
 [nan nan nan nan nan]]
7000 nan [[nan nan nan nan nan]
 [nan nan nan nan nan]
 [nan nan nan nan nan]
 [nan nan nan nan nan]
 [nan nan nan nan nan]]
7100 nan [[nan nan nan nan nan]
 [nan nan nan nan nan]
 [nan nan nan nan nan]
 [nan nan nan nan nan]
 [nan nan nan nan nan]]
7200 nan [[nan nan nan nan nan]
 [nan nan nan nan nan]
 [nan nan nan nan nan]
 [nan nan nan nan nan]
 [nan nan nan nan nan]]
7300 nan [[nan nan nan nan nan]
 [nan nan nan nan nan]
 [nan nan nan nan nan]
 [nan nan nan nan nan]
 [nan nan nan nan nan]]
7400 nan [[nan nan nan nan nan]
 [nan nan nan nan nan]
 [nan nan nan nan nan]
 [nan nan nan nan nan]
 [nan nan nan nan nan]]
7500 nan [[nan nan nan nan nan]
 [nan nan nan nan nan]
 [nan nan nan nan nan]
 [nan nan nan nan nan]
 [nan nan nan nan nan]]
7600 nan [[nan nan nan nan nan]
 [nan nan nan nan nan]
 [nan nan nan nan nan]
 [nan nan nan nan nan]
 [nan nan nan nan nan]]


## 고급 경사 하강법

```
    - 확률적 경사 하강법
    - 모멘텀      
```

In [2]:
learning_rate = 0.1
tf.set_random_seed(0)
np.random.seed(0)


Data_set =  np.loadtxt("./dataset/ThoraricSurgery.csv",delimiter=",", dtype=np.float32)
x_data = Data_set[:,0:17]
y_data = Data_set[:, [17]] 

X = tf.placeholder(tf.float32,[None,17])
Y = tf.placeholder(tf.float32,[None,1])

 
W1 = tf.Variable(tf.random_normal([17,30], name ='weight1'))
b1 = tf.Variable(tf.random_normal([30]), name = 'bias1')
layer1 = tf.nn.relu(tf.matmul(X,W1) + b1)   #hideen layer

W2 = tf.Variable(tf.random_normal([30,1]),name='weight2')
b2 = tf.Variable(tf.random_normal([1]),name='bias2')
hypothesis = tf.sigmoid(tf.matmul(layer1,W2) + b2)

cost = -tf.reduce_mean(Y*tf.log(hypothesis) + (1-Y) *tf.log(1-hypothesis))
train = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted,Y),dtype=tf.float32))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for step in range(10001):
        sess.run(train,feed_dict={X:x_data, Y:y_data})
        if step % 1000 == 0:
            print(step, sess.run(cost, feed_dict={X:x_data,Y:y_data}), sess.run(W2))
            
    h,c,a = sess.run([hypothesis, predicted, accuracy], feed_dict={X:x_data, Y:y_data})
    print("\nHypothesis:\n",h, "\nCorrect:\n",c ,"\nAccuracy:\n",a)

0 nan [[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]]
1000 nan [[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]]
2000 nan [[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]]
3000 nan [[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]]
4000 nan [[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [